In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import category_encoders as ce

In [ ]:
os.system('apt-get install p7zip')

In [ ]:
!apt install --assume-yes p7zip-full
!7z x ../input/mercari-price-suggestion-challenge/train.tsv.7z -y
!7z x ../input/mercari-price-suggestion-challenge/test.tsv.7z -y
!7z x ../input/mercari-price-suggestion-challenge/test_stg2.tsv.zip -y
!7z x ../input/mercari-price-suggestion-challenge/sample_submission.csv.7z -y
!7z x ../input/mercari-price-suggestion-challenge/sample_submission_stg2.csv.zip -y

In [ ]:
train = pd.read_csv("train.tsv",sep = '\t')
test = pd.read_csv("test.tsv",sep ='\t')

In [ ]:
train.shape

In [ ]:
test.shape

In [ ]:
train.head(60)

In [ ]:
print(train.keys())

In [ ]:
test.head(10)

In [ ]:
train['log_price'] = np.log1p(train.price)

In [ ]:
from scipy.stats import norm

fig, axes = plt.subplots(1, 3, figsize=(10,4))

axes[0].set_title('Price')
sns.distplot(train.price, ax=axes[0], kde=False)
axes[0].grid()

axes[1].set_title('Price < 75')
sns.distplot(train.price[train.price<75], ax=axes[1], kde=False)
axes[1].grid()

axes[2].set_title('log(Price + 1)')
sns.distplot(train.log_price, ax=axes[2], fit=norm, kde=False)
axes[2].set_xticks(range(0,9))
axes[2].grid()

fig.tight_layout()

In [ ]:
def plot_distribution_and_violin(variable):
    fig, axes = plt.subplots(2,1,figsize=(5,6), sharex=True)
    axes[0].set_title(variable)
    sns.countplot(x=variable, data=train, palette="ch:.25", color="c", ax=axes[0])
    sns.violinplot(x=variable, y='log_price', palette="ch:.25", data=train, ax=axes[1])
    fig.tight_layout()

In [ ]:
plot_distribution_and_violin('item_condition_id')

In [ ]:
plot_distribution_and_violin('shipping')
print('shipping - 1 if shipping fee is paid by seller and 0 by buyer')

In [ ]:
plot_distribution_and_violin('log_price')

In [ ]:
avg_log_price = train.groupby('shipping')['log_price'].mean()
diff = np.expm1(avg_log_price[0]) - np.expm1(avg_log_price[1])
print(f'Shipping shipping is paid by buyer ${diff:.3} greater than shipping is paid by seller')

In [ ]:
train[train.category_name.str.count('/')!=2].groupby('category_name').count()#sort_values(ascending=False)

In [ ]:
def transform_split_category_name(df):
    category_split = df['category_name'].str.split(r'/', n=2, expand=True)
    for i in [0,1,2]:
        df['cat_level_' + str(i)] = category_split[i]
    return df

In [ ]:
train.head()